In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import keras.layers.advanced_activations
import pickle
import sklearn

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation
from keras.layers import PReLU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 필요한 모듈들을 불러온다. 사용된 모듈은 numpy, pandas, keras, tensorflow, scikit-learn 이다.

In [2]:
print(sklearn.__version__)

0.23.1


In [3]:
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

# 랜덤변수를 생성할때 실행시마다 동일한 랜덤변수 생성을 위해 seed값을 지정해준다.

In [4]:
df = pd.read_csv('0_input_grade.csv', delimiter=',', encoding='euc_kr')
df.tail(50)

# 전처리한 데이터셋을 pandas로 불러온다.

,상권_구분_코드,상권_코드,서비스_업종_코드,매출단가,점포수,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,...,버스_정거장_수,등급,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68
97265,A,925,9,6343,11,37,63,2,42,29,...,3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97266,A,816,5,5705,14,65,35,3,52,26,...,2,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97267,D,1124,9,3433,5,68,32,34,34,7,...,2,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97268,A,906,10,7822,9,47,53,1,37,33,...,0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97269,D,1254,8,7040,13,33,67,8,40,25,...,11,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97270,A,305,10,5564,9,42,58,4,41,27,...,0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97271,A,467,10,5887,10,55,45,0,19,36,...,6,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97272,R,1330,5,10294,16,44,56,2,28,25,...,5,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97273,D,1049,10,7451,9,29,71,4,15,16,...,15,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97274,A,836,5,5415,7,55,45,4,60,22,...,1,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pkl_file = open('Departure_encoder.pkl', 'rb')
e = pickle.load(pkl_file) 
pkl_file.close()

# 사용자 환경에 관계없이 구동하기 위해 미리 fitting된 pickle 파일 형식의 LabelEncoder를 불러온다.

In [6]:
f_type = df['상권_구분_코드'].astype(str)
#e = LabelEncoder()
#e.fit(f_type)
a = e.transform(f_type)
print(a), len(a), len(df['상권_구분_코드'])

# 문자열인 상권_구분_코드를 scikit-learn의 LabelEncoder를 통해 정수로 치환해준다.

[1 0 0 ... 1 2 2]


(None, 97315, 97315)

In [7]:
#output = open('Departure_encoder.pkl', 'wb')
#pickle.dump(e, output)
#output.close()

# 위 줄에서 fitting한 LabelEncoder를 pickle 파일형식으로 저장한다.

In [8]:
df['상권_구분_코드'] = a

In [9]:
print(df['상권_구분_코드'])

0        1
1        0
2        0
3        0
4        1
        ..
97310    1
97311    2
97312    1
97313    2
97314    2
Name: 상권_구분_코드, Length: 97315, dtype: int32


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97315 entries, 0 to 97314
Data columns (total 69 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   상권_구분_코드           97315 non-null  int32  
 1   상권_코드              97315 non-null  int64  
 2   서비스_업종_코드          97315 non-null  int64  
 3   매출단가               97315 non-null  int64  
 4   점포수                97315 non-null  int64  
 5   남성_매출_비율           97315 non-null  int64  
 6   여성_매출_비율           97315 non-null  int64  
 7   연령대_10_매출_비율       97315 non-null  int64  
 8   연령대_20_매출_비율       97315 non-null  int64  
 9   연령대_30_매출_비율       97315 non-null  int64  
 10  연령대_40_매출_비율       97315 non-null  int64  
 11  연령대_50_매출_비율       97315 non-null  int64  
 12  연령대_60_이상_매출_비율    97315 non-null  int64  
 13  총_유동인구_수           97315 non-null  int64  
 14  남성_유동인구_수          97315 non-null  int64  
 15  여성_유동인구_수          97315 non-null  int64  
 16  연령대_10_유동인구_수      973

In [11]:
df.dropna(axis=1, inplace=True)

# NaN으로 채워진 열데이터를 dropna를 통해 제거해 준다.

In [12]:
dataset = df.values

In [13]:
pkl_file = open('Departure_encoder_1.pkl', 'rb')
e_1 = pickle.load(pkl_file) 
pkl_file.close()

# 사용자 환경에 관계없이 구동하기 위해 미리 fitting된 pickle 파일 형식의 LabelEncoder를 불러온다.

In [14]:
X = dataset[:,0:60]
Y = dataset[:,60]
f_type_1 = df['등급']
#e_1 = LabelEncoder()
#e_1.fit(f_type_1)
Y = e_1.transform(f_type_1)
Y_encoded = tf.keras.utils.to_categorical(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.1, random_state=seed)

# 전처리한 데이터를 속성값 X와 클래스 Y로 구분해주고 다중분류모델을 위해 클래스인 '등급'을 one-hot encoding 해준다.
# 이후 훈련셋과 테스트셋을 9대 1로 나누어준다.

In [15]:
#output = open('Departure_encoder_1.pkl', 'wb')
#pickle.dump(e_1, output)
#output.close()

# 위 줄에서 fitting한 LabelEncoder를 pickle 파일형식으로 저장한다.

In [16]:
print(Y_encoded[1])

[1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
scaler=StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)
# 빠른 학습과 보다 정확한 모델을 위해 Scikit-learn의 StandardScaler를 통해 데이터분포를 scaling해준다
# Scaling을 위해 fitting을 학습셋에 맞추고 그에 맞게 테스트셋을 scaling 해준다.

In [18]:
print(X_train_scale)

[[-0.68057342 -0.01343272 -1.30433545 ... -0.05142444 -0.34501279
  -0.60679854]
 [-0.68057342 -0.40796236 -1.30433545 ... -0.05142444 -0.34501279
  -0.80569092]
 [ 0.81164493  0.74396679 -1.30433545 ... -0.05142444 -0.34501279
  -0.01012141]
 ...
 [-0.68057342  0.57105565 -0.0784423  ... -0.05142444 -0.34501279
  -0.60679854]
 [-0.68057342 -1.12152524  1.14745086 ... -0.05142444  1.69614195
  -0.20901379]
 [-0.68057342 -0.07188155 -0.0784423  ... -0.05142444 -0.34501279
  -0.40790616]]


In [19]:
#model = Sequential()
#model.add(Dense(256, input_dim=60))
#model.add(BatchNormalization())
#model.add(PReLU())
#model.add(Dense(256))
#model.add(BatchNormalization())
#model.add(PReLU())
#model.add(Dense(9))
#model.add(BatchNormalization())
#model.add(Activation('softmax'))
#model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0001),metrics=['accuracy'])
#model.fit(X_train_scale, Y_train, epochs=1000, batch_size=4096)

# 각 은닉층마다 512개의 노드를 가지고 있으며 2개의 은닉층으로 딥러닝모델을 설계하였다.
# 활성화함수는 Dying ReLU현상을 막기 위해 PReLU를 사용하였다.
# 다중분류모델을 위해 출력층의 활성화함수는 softmax로 설계하고 손실함수는 categorical_crossentropy로 하였다.
# 최적화함수는 Adam으로 설정하였다. 학습과정에서 정확도를 위해 learning rate인 lr과 batch_size를 중간중간 가변적으로 변경하였다.
# 1~9등급으로 분류되므로 출력층의 노드개수는 9개이다.
# 학습이 완료된 모델은 'grade_predict.h5' 파일로 저장하였고 다음 코드에서 불러왔다.

In [20]:
model = load_model('grade_predict.h5')

# 저장된 모델을 불러온다.

In [21]:
model.evaluate(X_test_scale,Y_test)[1]

# 모델을 테스트셋으로 평가한다. 정확도는 88.81%임을 알 수 있다.

305/305 [==============================] - 0s 663us/step - loss: 0.3296 - accuracy: 0.8858


0.8858405351638794

In [22]:
df1 = pd.read_csv('test_input_grade.csv', delimiter=',', encoding='euc_kr')

# 실제 공공자전거 누적경로로 추정한 푸드트럭 위치에 대한 후보 데이터를 불러온다.

In [23]:
df1.head()

,상권_구분_코드,상권_코드,서비스_업종_코드,매출단가,점포수,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,...,대학교_수,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수
0,A,55,9,3000,3,52,48,0,33,29,...,0,0,1,0,0,0,0,0,0,1
1,A,55,1,3000,12,72,28,1,20,26,...,0,0,1,0,0,0,0,0,0,1
2,A,55,9,4000,3,52,48,0,33,29,...,0,0,1,0,0,0,0,0,0,1
3,A,55,1,4000,12,72,28,1,20,26,...,0,0,1,0,0,0,0,0,0,1
4,A,55,9,5000,3,52,48,0,33,29,...,0,0,1,0,0,0,0,0,0,1


In [24]:
pkl_file = open('Departure_encoder_1.pkl', 'rb')
e_1 = pickle.load(pkl_file) 
pkl_file.close()

In [25]:
f_type_2 = df1['상권_구분_코드'].astype(str)
a1 = e.transform(f_type_2)
df1['상권_구분_코드'] = a1
print(df1['상권_구분_코드'])

# 위에서와 마찬가지로 문자열데이터인 '상권_구분_코드' 열데이터를 LabelEncoding 해준다.

0       0
1       0
2       0
3       0
4       0
       ..
1303    2
1304    2
1305    2
1306    2
1307    2
Name: 상권_구분_코드, Length: 1308, dtype: int32


In [26]:
output = open('Departure_encoder_1.pkl', 'wb')
pickle.dump(e_1, output)
output.close()

In [27]:
dataset1=df1.values

In [28]:
X_a = dataset1[:,0:60]

In [29]:
X_a

array([[   0,   55,    9, ...,    0,    0,    1],
       [   0,   55,    1, ...,    0,    0,    1],
       [   0,   55,    9, ...,    0,    0,    1],
       ...,
       [   2, 1316,    9, ...,    0,    0,    7],
       [   2, 1316,    5, ...,    0,    0,    7],
       [   2, 1316,    1, ...,    0,    0,    7]], dtype=int64)

In [30]:
X_a_scale = scaler.transform(X_a)

# 훈련셋으로 fitting한 scaler로 해당 데이터를 scaling 한다.

In [31]:
predictions = model.predict(X_a_scale)
result = []
for i in range(len(X_a)):
    predictions[i].shape 
    np.sum(predictions[i])
    result.append(np.argmax(predictions[i]))
    
# 모델을 통해 예측한 등급별 확률중 가장 높은 확률을 1로 나머지 확률은 0으로 argmax를 통해 치환하여 매출등급을 도출한다.
# one-hot encoding으로 분류되었기 때문에 0번째가 1인경우 1등급 1번째가 1인경우 2등급으로 분류되었다.
# 예시1 : [0.6 0.05 0.05 0.05 0.05 0.05 0.05 0.05 0.05] -> [1 0 0 0 0 0 0 0 0] -> 0 (1등급)
# 예시2 : [0.05 0.6 0.05 0.05 0.05 0.05 0.05 0.05 0.05] -> [0 1 0 0 0 0 0 0 0] -> 1 (2등급)

In [32]:
print(result)

[7, 6, 7, 6, 7, 6, 7, 6, 7, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 5, 4, 6, 5, 4, 6, 5, 4, 6, 5, 4, 6, 5, 4, 6, 5, 4, 6, 4, 6, 8, 6, 8, 6, 8, 6, 8, 6, 8, 8, 1, 6, 8, 1, 6, 8, 1, 6, 8, 1, 6, 8, 1, 6, 8, 1, 6, 8, 6, 8, 6, 8, 6, 8, 6, 8, 6, 6, 6, 7, 7, 7, 7, 3, 7, 3, 7, 3, 7, 3, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 8, 4, 8, 3, 6, 3, 4, 8, 3, 6, 3, 4, 2, 3, 6, 3, 4, 2, 3, 6, 2, 4, 2, 3, 5, 2, 4, 2, 3, 7, 8, 8, 7, 8, 8, 7, 8, 8, 7, 8, 8, 7, 8, 8, 7, 8, 8, 4, 8, 8, 4, 8, 8, 4, 8, 8, 4, 8, 8, 4, 8, 8, 4, 8, 8, 8, 7, 8, 8, 7, 8, 8, 7, 8, 8, 7, 8, 8, 1, 8, 8, 1, 8, 1, 2, 5, 1, 2, 5, 1, 2, 2, 1, 2, 2, 1, 2, 2, 4, 2, 2, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 8, 4, 4, 6, 4, 8, 5, 8, 4, 8, 5, 5, 4, 8, 5, 5, 5, 8, 5, 5, 5, 8, 3, 5, 5, 8, 3, 5, 5, 2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 4, 2, 2, 4, 3, 6, 3, 6, 2, 6, 2, 6, 2, 6, 2, 6, 8, 6, 7, 8, 6, 8, 6, 7, 8, 6, 8, 6, 4, 8, 6, 8, 6, 4, 8, 6, 8, 6, 4, 8, 6, 8, 6, 4, 8, 6, 7, 7, 7, 7, 7, 7, 6, 8, 6, 1, 8, 6, 1, 8, 6, 1, 8, 6, 1, 8, 6, 1, 2, 6, 8, 8, 5, 

In [33]:
a = np.array(result)
print(a)

[7 6 7 ... 4 5 6]


In [34]:
df1['등급']=a+1

# 등급은 result값에 1을 더한 값이므로 result list의 모든 원소에 1을 더하였다.

In [35]:
df1.head()

,상권_구분_코드,상권_코드,서비스_업종_코드,매출단가,점포수,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,...,백화점_수,슈퍼마켓_수,극장_수,숙박_시설_수,공항_수,철도_역_수,버스_터미널_수,지하철_역_수,버스_정거장_수,등급
0,0,55,9,3000,3,52,48,0,33,29,...,0,1,0,0,0,0,0,0,1,8
1,0,55,1,3000,12,72,28,1,20,26,...,0,1,0,0,0,0,0,0,1,7
2,0,55,9,4000,3,52,48,0,33,29,...,0,1,0,0,0,0,0,0,1,8
3,0,55,1,4000,12,72,28,1,20,26,...,0,1,0,0,0,0,0,0,1,7
4,0,55,9,5000,3,52,48,0,33,29,...,0,1,0,0,0,0,0,0,1,8


In [36]:
df1.to_csv('test_with_grade.csv',encoding='euc_kr')

# 등급에 대한 열을 추가하여 csv파일로 저장한다.